In [1]:
import pandas as pd
import re
import numpy as np
import textstat
import random
from collections import Counter
from stargazer.stargazer import Stargazer
import statsmodels.api as sm
import matplotlib.pyplot as plt
# Temporarily adjust option to display up to 500 columns 
pd.set_option('display.max_info_columns', 500)

# Import the filtered file

In [2]:
df=pd.read_excel('D:/trykicstarter/Kickstarter_0419/data/2.Filtered/kickstarter_filtered_usd_v4.xlsx')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54763 entries, 0 to 54762
Data columns (total 44 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Unnamed: 0             54763 non-null  int64         
 1   ProjectID              54763 non-null  int64         
 2   Subcategories          54763 non-null  object        
 3   URL                    54763 non-null  object        
 4   State                  54763 non-null  bool          
 5   ProjectName            54762 non-null  object        
 6   ProjectDescription     54714 non-null  object        
 7   CreatorName            54749 non-null  object        
 8   LastUpdate             54762 non-null  datetime64[ns]
 9   UpdateCount            54763 non-null  int64         
 10  CommentCount           54763 non-null  int64         
 11  ProWeLove              54763 non-null  bool          
 12  ProjectLocation        54763 non-null  object        
 13  P

# Generate Education related columns

## Generate columns named Education_Context(the Educaitonal Context form Biography)  ,High_Educated_Bio(a boolean indicator (True/False) of whether the biography contains any education-related context),Eductional_level( the level of the education received,the higher ,the smaller

In [35]:
# # Define the full and expanded list of education-related keywords
# education_keywords = [
#   # Specific terms
#     r'\bgrad\b', 'graduate', 'degree', 'alumnus', 'alumna', 'graduand', r'\diploma\b',
#     'educated', 'educating', 'graduation', 'graduating',
#     # Institutions
#     'academy', 'institute', 'university', 'college', r'\bUniv\b', 
#     # Degree abbreviations with word boundaries bachelor\'s',
#     'bachelor',  'undergraduate' , r'\bundergra\b', r'\bundergrad\b',
#     r'\bba\b', r'\bbsc\b', r'\bbba\b', r'\bbbs\b', r'\bb\.a\.\b', r'\bb\.?S\.?\b', r'\bbfa\b', r'\bbacc\b', r'\bBASc\b', r'\bBOptom\b',
#     r'\bBDS\b', r'\bBDes\b', r'\bBD\b', r'\bBScEcon\b',  r'\bBEng\b', r'\bBFin\b', r'\bBHSc\b', r'\bLLB\b', r'\bBMedSci\b',
#     r'\bBM\b', r'\bBMBS\b', r'\bBMus\b', r'\bBNurs\b', r'\bBSc\b', r'\bBSocSc\b', r'\bBSW\b', r'\bBTech\b', r'\bBPharm\b', r'\bBLitt\b',
#     r'\bBCL\b', r'\bLLM\b',   r'\bOD\b', r'\bDVM\b', r'\bPharmD\b',
    
#     r'\bmaster\b', r'\bmasters\b', 'mphil', 'postgraduate', r'\bms\b', r'\bm\.b\.a\.\b', r'\bma\b', r'\bm\.a\.\b', r'\bm\.s\.\b',
#     r'\bMSc\b', r'\bMEng\b', r'\bMEd\b', r'\bMFA\b', r'\bMBA\b', r'\bMPhil\b', r'\bLLM\b', r'\bMPA\b', r'\bMPP\b', r'\bMSW\b',
#     r'\bMTech\b', r'\bMPharm\b', r'\bMRes\b', r'\bMLitt\b',
#     r'\bPh\.?d\b', r'\bdr\b', r'\bdoctor\b', r'\bdoctoral\b', 'doctorate', r'\bpostdoc\b', 'postdoctoral',  'dphil','Post Doc',
#     # Other educational terms     'summa cum laude', 'magna cum laude',
#     'cum laude', 'dean\'s list','dissertation','thesis'
# ]

# # Adjusting the function to identify and return the specific matching keywords
# def identify_matching_keywords(sentence, keywords):
#     matches = []
#     # Check if sentence is a string
#     if isinstance(sentence, str):
#         for keyword in keywords:
#             if re.search(keyword, sentence, re.IGNORECASE):
#                 matches.append(keyword)
#         # Expanded pattern to match various phrases indicating education at an institution
#        # Updated pattern
#         education_pattern = r'\b(student|graduated|graduate|graduating|go to|went to|earned a degree|earning a degree|received a degree|holds a degree|receive a degree|studying|studied|enrolled|enrolling|alumni|education|education background|academic background)\b\s+(in|at|from)\b.*?(\b[A-Z][\w\s]*(University|College|Institute|School|Academy|Tech)\b)'



#         education_matches = re.findall(education_pattern, sentence, re.IGNORECASE)
#         matches.extend([match[2] for match in education_matches])
#     return matches#
# #'bachelor', 


In [82]:
education_keywords = [#r'\bgraduat\w*',r'\balumn\w*',
    r'\bgrad\b', 'degree', 'graduand', r'\bdiploma\b',r'\bbachelor',
    'educated', 'educating', 'graduat','alumn',
    'academy', 'institute', 'university', 'college', r'\bUniv\b',
    'undergraduate', r'\bundergra\b', r'\bundergrad\b',
    r'\bba\b', r'\bbsc\b', r'\bbba\b', r'\bbbs\b', r'\bb\.a\.\b', r'\bb\.?S\.?\b', r'\bbfa\b', r'\bbacc\b', r'\bBASc\b', r'\bBOptom\b',
    r'\bBDS\b', r'\bBDes\b', r'\bBD\b', r'\bBScEcon\b', r'\bBEng\b', r'\bBFin\b', r'\bBHSc\b', r'\bLLB\b', r'\bBMedSci\b',
    r'\bBM\b', r'\bBMBS\b', r'\bBMus\b', r'\bBNurs\b', r'\bBSc\b', r'\bBSocSc\b', r'\bBSW\b', r'\bBTech\b', r'\bBPharm\b', r'\bBLitt\b',
    r'\bBCL\b', r'\bLLM\b', r'\bOD\b', r'\bDVM\b', r'\bPharmD\b',
    r'\bmaster\b', r'\bmasters\b', 'mphil', 'postgraduate', r'\bms\b', r'\bm\.b\.a\.\b', r'\bma\b', r'\bm\.a\.\b', r'\bm\.s\.\b',r'\bm\.s\.c\.\b',
    r'\bMSc\b', r'\bMEng\b', r'\bMEd\b', r'\bMFA\b',r'\bMFE\b', r'\bMBA\b', r'\bMPhil\b', r'\bLLM\b', r'\bMPA\b', r'\bMPP\b', r'\bMSW\b',
    r'\bMTech\b', r'\bMPharm\b', r'\bMRes\b', r'\bMLitt\b',
    r'\bPh\.?d\b', r'\bdr\b', r'\bdoctor\b', r'\bdoctoral\b', 'doctorate', r'\bpostdoc\b', 'postdoctoral', 'dphil', 'Post Doc',
    'summa cum laude', 'magna cum laude', 'cum laude', 'dean\'s list', 'dissertation', 'thesis'
]

def identify_matching_keywords(sentence, keywords):
    matches = []
    positions = []

    for keyword in keywords:
        for match in re.finditer(keyword, sentence, re.IGNORECASE):
            start, end = match.span()
            if not any(start >= p and end <= q for p, q in positions):
                matches.append(match.group())
                positions.append((start, end))

    return matches


In [83]:
# Function to test sentences
def test_sentences(sentences):
    for i, sentence in enumerate(sentences, start=1):
        matches = identify_matching_keywords(sentence, education_keywords)
        print(f"Sentence {i}: {sentence}")
        print("Matching keywords:", matches)
        print()

# Test sentences
sentences_to_test = ["John is a university student in USA.John graduated from Harvard University with a bachelor's degree in Computer Science.",
                     "John studied in MIT with a bachelor degree in Computer Science.",
                     "Samantha is currently studying for her master's at Stanford.",
                     "David received his Ph.D. from MIT.",
                     "Emily holds a degree in Economics from Yale University.",
                     "Mark is an alumnus of Oxford University.",
                     "Lucy is a postdoctoral researcher at Cambridge.",
                     "Jennifer earned her mba from Wharton Business School. She also has a M.S in Applied Psychology",
                     "Join 10 Months Part-time Post Doc Singapore.",
                     "Michael's dissertation focused on renewable energy.",
                     "Rachel is a magna cum laude graduate.B.S in Chemistry.",
                     "Alex is enrolling in a bachelor's program at UCLA,then plan to hold a MFE there.",
                     'I am a 24 year old graduate from Grand Valley State University. It was there where I earned my B.B.A. in Finance and birthed my passion for business.'
                     'I am driving a car',
                     'I mastered in the paiting',
                     'THe school has many students'
                    ]

test_sentences(sentences_to_test)

Sentence 1: John is a university student in USA.John graduated from Harvard University with a bachelor's degree in Computer Science.
Matching keywords: ['degree', 'bachelor', 'graduat', 'university', 'University']

Sentence 2: John studied in MIT with a bachelor degree in Computer Science.
Matching keywords: ['degree', 'bachelor']

Sentence 3: Samantha is currently studying for her master's at Stanford.
Matching keywords: ['master']

Sentence 4: David received his Ph.D. from MIT.
Matching keywords: ['Ph.D']

Sentence 5: Emily holds a degree in Economics from Yale University.
Matching keywords: ['degree', 'University']

Sentence 6: Mark is an alumnus of Oxford University.
Matching keywords: ['alumn', 'University']

Sentence 7: Lucy is a postdoctoral researcher at Cambridge.
Matching keywords: ['postdoctoral']

Sentence 8: Jennifer earned her mba from Wharton Business School. She also has a M.S in Applied Psychology
Matching keywords: ['mba']

Sentence 9: Join 10 Months Part-time Post Do

In [84]:
# Convert the 'Biography' column to string
df['Biography'] = df['Biography'].astype(str)

# Apply the identify_matching_keywords function
df['Education_Context'] = df['Biography'].apply(identify_matching_keywords, args=(education_keywords,))
df['High_Educated_Bio'] = df['Education_Context'].apply(lambda x: bool(x))


In [85]:
df['High_Educated_Bio'].sum()

9598

In [86]:
def get_keyword_stats(education_context):
        # Convert all keywords to lowercase
    education_context = [kw.lower() for kw in education_context]

    # Get keyword occurrence
    keyword_occurrence = Counter(education_context)
    
    # Get unique keywords
    unique_keywords = list(keyword_occurrence.keys())
    
    # Get total appearance of keywords
    total_appearance = sum(keyword_occurrence.values())
    
    # Get word count
    word_count = sum(len(keyword.split()) for keyword in education_context)
    
    return keyword_occurrence, unique_keywords, total_appearance, word_count


In [87]:
# Apply the get_keyword_stats function
df['Edu_Keywords_Occurrence'], df['Edu_Unique_Keywords'], df['Edu_Total_Appearance'], df['Edu_Word_Count'] = zip(*df['Education_Context'].apply(get_keyword_stats))


In [88]:
df[['Education_Context', 'Edu_Keywords_Occurrence', 'Edu_Unique_Keywords', 'Edu_Total_Appearance', 'Edu_Word_Count']].head(50)

,Education_Context,Edu_Keywords_Occurrence,Edu_Unique_Keywords,Edu_Total_Appearance,Edu_Word_Count
0,[],{},[],0,0
1,[],{},[],0,0
2,[],{},[],0,0
3,[degree],{'degree': 1},[degree],1,1
4,[],{},[],0,0
5,[],{},[],0,0
6,[educated],{'educated': 1},[educated],1,1
7,[],{},[],0,0
8,[],{},[],0,0
9,[],{},[],0,0


In [99]:
def extract_education_level(keywords_list):
    # Define categories for different educational levels
    doctorate_keywords = [ r'\bPh\.?d\b', r'\bdr\b', r'\bdoctor\b', r'\bdoctoral\b', 'doctorate', r'\bpostdoc\b', 'postdoctoral', 'dphil', 'Post Doc']
    graduate_keywords = [r'\bmaster\b', r'\bmasters\b', 'mphil', 'postgraduate', r'\bms\b', r'\bmba\b', r'\bm\.b\.a\.\b', r'\bma\b', r'\bm\.a\.\b', r'\bm\.s\.\b',
                         r'\bMSc\b', r'\bMEng\b', r'\bMEd\b', r'\bMFA\b', r'\bMFE\b', r'\bMBA\b', r'\bMPhil\b', r'\bLLM\b', r'\bMPA\b', r'\bMPP\b', r'\bMSW\b',
                         r'\bMTech\b', r'\bMPharm\b', r'\bMRes\b', r'\bMLitt\b']
    bachelors_keywords = [
        r'\bgrad\b', 'degree',  'graduand', r'\diploma\b','graduat','alumn',
        'educated', 'educating',
        'academy', 'institute', 'university', 'college', r'\bUniv\b',
        'bachelor', 'undergraduate', r'\bundergra\b', r'\bundergrad\b',
        r'\bba\b', r'\bbsc\b', r'\bbba\b', r'\bbbs\b', r'\bb\.a\.\b', r'\bb\.?S\.?\b', r'\bbfa\b', r'\bbacc\b', r'\bBASc\b', r'\bBOptom\b',
        r'\bBDS\b', r'\bBDes\b', r'\bBD\b', r'\bBScEcon\b', r'\bBEd\b', r'\bBEng\b', r'\bBFin\b', r'\bBHSc\b', r'\bLLB\b', r'\bBMedSci\b',
        r'\bBM\b', r'\bBMBS\b', r'\bBMus\b', r'\bBNurs\b', r'\bBSc\b', r'\bBSocSc\b', r'\bBSW\b', r'\bBTech\b', r'\bBPharm\b', r'\bBLitt\b',
        r'\bBCL\b', r'\bLLM\b', r'\bMD\b', r'\bDC\b', r'\bOD\b', r'\bDVM\b', r'\bPharmD\b',
        'summa cum laude', 'magna cum laude', 'cum laude', 'dean\'s list', 'dissertation', 'thesis'
    ]

    # Check for matches starting from the highest level
    if any(keyword in keywords_list for keyword in doctorate_keywords):
        return 'Doctorate'  # Doctorate level
    elif any(keyword in keywords_list for keyword in graduate_keywords):
        return 'Graduate'  # Graduate level
    elif any(keyword in keywords_list for keyword in bachelors_keywords):
        return 'Bachelors'  # Bachelor's level
    else:
        return 'Below Bachelors'  # High school level or lower / no match

# Applying the function to the DataFrame
df['Education_Level'] = df['Education_Context'].apply(extract_education_level)


In [100]:
df['Education_Level'].unique()

array(['Below Bachelors', 'Bachelors', 'Doctorate', 'Graduate'],
      dtype=object)

In [101]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54763 entries, 0 to 54762
Data columns (total 52 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Unnamed: 0               54763 non-null  int64         
 1   ProjectID                54763 non-null  int64         
 2   Subcategories            54763 non-null  object        
 3   URL                      54763 non-null  object        
 4   State                    54763 non-null  bool          
 5   ProjectName              54762 non-null  object        
 6   ProjectDescription       54714 non-null  object        
 7   CreatorName              54749 non-null  object        
 8   LastUpdate               54762 non-null  datetime64[ns]
 9   UpdateCount              54763 non-null  int64         
 10  CommentCount             54763 non-null  int64         
 11  ProWeLove                54763 non-null  bool          
 12  ProjectLocation          54763 n

# Testing

In [102]:
# Specified columns
columns = ['ProjectID', 'Biography', 'Education_Context', 'High_Educated_Bio', 'Education_Level', 'Edu_Keywords_Occurrence', 'Edu_Unique_Keywords', 'Edu_Total_Appearance', 'Edu_Word_Count']

# Check if specified columns exist in df
if all(col in df.columns for col in columns):
    # Filter the DataFrame for the specific Project ID (e.g., Project ID 24243)
    project_id = 5941
    project_data = df[df['ProjectID'] == project_id][columns]

    if not project_data.empty:
        # Print details of the project
        row = project_data.iloc[0]  # Assuming there's only one matching row

        # Print details of the project
        print(f"Project ID: {row['ProjectID']}\n"
              f"Biography: {row['Biography']}\n"
              f"Education Context: {row['Education_Context']}\n"
              f"High Educated Bio: {row['High_Educated_Bio']}\n"
              f"Education Level: {row['Education_Level']}\n"
              f"Edu Keywords Occurrence: {row['Edu_Keywords_Occurrence']}\n"
              f"Edu Unique Keywords: {row['Edu_Unique_Keywords']}\n"
              f"Edu Total Appearance: {row['Edu_Total_Appearance']}\n"
              f"Edu Word Count: {row['Edu_Word_Count']}")
    else:
        print("No data found for the specified Project ID.")
else:
    print("Some specified columns do not exist in the DataFrame.")


Project ID: 5941
Biography: Creator



The Roof Duster project is led by Wayne Dehart.

Forty year resident of San Antonio, Tx, married with children.

Holds a BS in Mathematics.

Held various positions IT capacities including software dev and mgmt.

Loves brown rice, herbal shampoo, and peace signs.

Ardent hater of roof algae stains.

Hey, I've changed the upper and lower O-rings on a 2005 Jaguar S-Type intake manifold (it's the lower O-rings that separate Grasshopper from Master).  We can build this.
Education Context: ['BS', 'Master']
High Educated Bio: True
Education Level: Below Bachelors
Edu Keywords Occurrence: Counter({'bs': 1, 'master': 1})
Edu Unique Keywords: ['bs', 'master']
Edu Total Appearance: 2
Edu Word Count: 2


In [103]:
# Specified columns
columns = ['ProjectID', 'Biography', 'Education_Context', 'High_Educated_Bio', 'Education_Level', 'Edu_Keywords_Occurrence', 'Edu_Unique_Keywords', 'Edu_Total_Appearance', 'Edu_Word_Count']

# Check if specified columns exist in df
if all(col in df.columns for col in columns):
    # Filter the DataFrame for the specific Project ID (e.g., Project ID 24243)
    project_id = 8192
    project_data = df[df['ProjectID'] == project_id][columns]

    if not project_data.empty:
        # Print details of the project
        row = project_data.iloc[0]  # Assuming there's only one matching row

        # Print details of the project
        print(f"Project ID: {row['ProjectID']}\n"
              f"Biography: {row['Biography']}\n"
              f"Education Context: {row['Education_Context']}\n"
              f"High Educated Bio: {row['High_Educated_Bio']}\n"
              f"Education Level: {row['Education_Level']}\n"
              f"Edu Keywords Occurrence: {row['Edu_Keywords_Occurrence']}\n"
              f"Edu Unique Keywords: {row['Edu_Unique_Keywords']}\n"
              f"Edu Total Appearance: {row['Edu_Total_Appearance']}\n"
              f"Edu Word Count: {row['Edu_Word_Count']}")
    else:
        print("No data found for the specified Project ID.")
else:
    print("Some specified columns do not exist in the DataFrame.")


Project ID: 8192
Biography: 2-time creator



We are a team of three engineers working to make affordable, self-managed safe renewable energy for your home. 

James Zinke 
A recognized technology and customer service leader known for his strategic and innovative vision; James offers a unique blend of executive acumen, customer centric IT solution development. With a reputation for professionalism, judicious use of resources, and cost containment, James’s work has been instrumental in providing Washington Mutual Bank, J.P Morgan Bank and Chase Bank with streamed lined and cost effective customer centric solutions.

James is always on the go. After being recruited on to University of the Ozarks Cross Country Running Team, James has been avid cross-country runner. You can catch him getting his daily 5 miles most mornings. He is also an avid participant in Racquetball tournaments, Charity 5k Fun Runs, and has a pretty respectable background in Fencing, Paragliding, and culinary cooking. 



In [104]:
# Specified columns
columns = ['ProjectID', 'Biography', 'Education_Context', 'High_Educated_Bio', 'Education_Level', 'Edu_Keywords_Occurrence', 'Edu_Unique_Keywords', 'Edu_Total_Appearance', 'Edu_Word_Count']

# Check if specified columns exist in df
if all(col in df.columns for col in columns):
    # Filter the DataFrame for the specific Project ID (e.g., Project ID 24243)
    project_id = 6464
    project_data = df[df['ProjectID'] == project_id][columns]

    if not project_data.empty:
        # Print details of the project
        row = project_data.iloc[0]  # Assuming there's only one matching row

        # Print details of the project
        print(f"Project ID: {row['ProjectID']}\n"
              f"Biography: {row['Biography']}\n"
              f"Education Context: {row['Education_Context']}\n"
              f"High Educated Bio: {row['High_Educated_Bio']}\n"
              f"Education Level: {row['Education_Level']}\n"
              f"Edu Keywords Occurrence: {row['Edu_Keywords_Occurrence']}\n"
              f"Edu Unique Keywords: {row['Edu_Unique_Keywords']}\n"
              f"Edu Total Appearance: {row['Edu_Total_Appearance']}\n"
              f"Edu Word Count: {row['Edu_Word_Count']}")
    else:
        print("No data found for the specified Project ID.")
else:
    print("Some specified columns do not exist in the DataFrame.")


Project ID: 6464
Biography: 3-time creator



I am a 24 year old graduate from Grand Valley State University. It was there where I earned my B.B.A. in Finance and birthed my passion for business. After my time at GVSU, I enrolled at Michigan State University's College of Law. Four months after starting, I decided to change route and abandon law school to focus on my dream of starting my own company. I am a firm believer of perspective; I believe that the more perspective I have, the more I will be able to relate to my consumer base. Every business, no matter the industry, has the same end product: people. If you can relate to people, there is no reason that you can't develop a product and/or company that meets their needs. More importantly, if you can relate to people, you can also develop a company that meets the needs of its employees. A happy employee means a happy product, and a happy product means a happy customer. Success starts in-house, something I think a lot of companies over

In [105]:
# Specified columns
columns = ['ProjectID', 'Biography', 'Education_Context', 'High_Educated_Bio', 'Education_Level', 'Edu_Keywords_Occurrence', 'Edu_Unique_Keywords', 'Edu_Total_Appearance', 'Edu_Word_Count']

# Check if specified columns exist in df
if all(col in df.columns for col in columns):
    # Randomly select a row (project) from the DataFrame
    random_index = random.choice(df.index)
    row = df.loc[random_index]  # Select the row by index

    # Print details of the randomly selected project
    print(f"Project ID: {row['ProjectID']}\n"
          f"Biography: {row['Biography']}\n"
          f"Education Context: {row['Education_Context']}\n"
          f"High Educated Bio: {row['High_Educated_Bio']}\n"
          f"Education Level: {row['Education_Level']}\n"
          f"Edu Keywords Occurrence: {row['Edu_Keywords_Occurrence']}\n"
          f"Edu Unique Keywords: {row['Edu_Unique_Keywords']}\n"
          f"Edu Total Appearance: {row['Edu_Total_Appearance']}\n"
          f"Edu Word Count: {row['Edu_Word_Count']}")
else:
    print("Some specified columns do not exist in the DataFrame.")



Project ID: 32162
Biography: Creator



Zach is the founder of Down2aScience. He is a creator, artist, puzzler and snowboarder. At Down2aScience, we enjoy finding creative solutions to every day problems.
Education Context: []
High Educated Bio: False
Education Level: Below Bachelors
Edu Keywords Occurrence: Counter()
Edu Unique Keywords: []
Edu Total Appearance: 0
Edu Word Count: 0


# Create a list for midel school

In [106]:
middle_school_keywords = [
    'middle school',
    'high school',
    'junior high',
    'senior high',
    'secondary school',
    'junior secondary',
    'secondary education',
    'junior school',
    'nursery school',
]

def identify_matching_keywords(sentence, keywords):
    matches = []
    positions = []

    for keyword in keywords:
        for match in re.finditer(keyword, sentence, re.IGNORECASE):
            start, end = match.span()
            if not any(start >= p and end <= q for p, q in positions):
                matches.append(match.group())
                positions.append((start, end))

    return matches

# Apply the identify_matching_keywords function for middle school
df['Middle_School_Context'] = df['Biography'].apply(identify_matching_keywords, args=(middle_school_keywords,))


In [107]:
df['Middle_School_Context'].head(50)

0                  []
1                  []
2                  []
3                  []
4                  []
5                  []
6                  []
7                  []
8                  []
9                  []
10                 []
11                 []
12                 []
13                 []
14                 []
15                 []
16                 []
17                 []
18                 []
19                 []
20                 []
21                 []
22                 []
23                 []
24                 []
25                 []
26                 []
27                 []
28                 []
29                 []
30                 []
31                 []
32                 []
33                 []
34                 []
35                 []
36                 []
37                 []
38                 []
39                 []
40                 []
41      [High School]
42                 []
43                 []
44    [Middle School]
45        

In [108]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54763 entries, 0 to 54762
Data columns (total 52 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Unnamed: 0               54763 non-null  int64         
 1   ProjectID                54763 non-null  int64         
 2   Subcategories            54763 non-null  object        
 3   URL                      54763 non-null  object        
 4   State                    54763 non-null  bool          
 5   ProjectName              54762 non-null  object        
 6   ProjectDescription       54714 non-null  object        
 7   CreatorName              54749 non-null  object        
 8   LastUpdate               54762 non-null  datetime64[ns]
 9   UpdateCount              54763 non-null  int64         
 10  CommentCount             54763 non-null  int64         
 11  ProWeLove                54763 non-null  bool          
 12  ProjectLocation          54763 n

In [109]:
df.columns

Index(['Unnamed: 0', 'ProjectID', 'Subcategories', 'URL', 'State',
       'ProjectName', 'ProjectDescription', 'CreatorName', 'LastUpdate',
       'UpdateCount', 'CommentCount', 'ProWeLove', 'ProjectLocation',
       'Pledged', 'Goal', 'ImgCount', 'videoCount', 'youtubeCount', 'Backer',
       'StartDate', 'EndDate', 'Days', 'Featured', 'Story', 'CreatorLocation',
       'Verified', 'CreatorURL', 'Connect', 'Text', 'Biography', 'backed',
       'joined', 'create', 'webList', 'Category', 'WebNum', 'IsFeatured',
       'IsConnect', 'TotalCreated', 'TotalBacked', 'IsIdentityVerified',
       'IsVerifiedIDAvailable', 'Pledged_USD', 'Goal_USD', 'Education_Context',
       'High_Educated_Bio', 'Edu_Keywords_Occurrence', 'Edu_Unique_Keywords',
       'Edu_Total_Appearance', 'Edu_Word_Count', 'Education_Level',
       'Middle_School_Context'],
      dtype='object')

# Create a final file with key variables

In [110]:
# List of key variables to retain for regression analysis
key_variables = [
    'ProjectID', 'Education_Context',
       'High_Educated_Bio', 'Education_Level', 'Edu_Unique_Keywords',
       'Edu_Total_Appearance', 'Edu_Word_Count', 'Edu_Keywords_Occurrence','Middle_School_Context'
    # Add any other key variables you need
]

# Create a new DataFrame with only the key variables
df_final = df[key_variables]

In [111]:
# Export the streamlined DataFrame to an Excel file for use in analysis+
df_final.to_excel('D:/trykicstarter/Kickstarter_0419/data/3.Final/kickstarter_education_usd_0428.xlsx', index=False)